In [1]:
import pandas as pd
from datetime import timedelta
import numpy as np
import os
from collections import Counter
pd.set_option("display.precision", 8)

# Paxful

In [2]:
trades_paxful = pd.read_csv('../temporary/trades_paxful_cleaned.csv',
                            dtype={
                                'tid':int,
                                'amount':float,
                                'price':float,
                                'crypto_rate_usd':float,
                                'amount_usd':float
                            },
                            parse_dates = ['date'],
                            infer_datetime_format = True
                           )

In [4]:
trades_paxful.head()

,tid,date,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,crypto_rate_usd,amount_usd
0,4544629,2017-03-15 06:04:04,0.01306959,20.0,"Any Visa, MasterCard and AmEx Gift Cards",Gift cards,USD,buy,US,US,1246.86,16.29594899
1,4544627,2017-03-15 06:04:26,0.05728689,100.0,PayPal,Online wallets,USD,buy,CM,US,1246.86,71.42873167
2,4544639,2017-03-15 06:04:39,0.01688459,25.0,Amazon Gift Card,Gift cards,USD,buy,US,US,1247.49,21.06335718
3,4544613,2017-03-15 06:05:27,0.09403504,150.0,iTunes Gift Card,Gift cards,USD,buy,CN,ZA,1246.86,117.24852997
4,4544617,2017-03-15 06:05:28,0.59586015,1300.0,Amazon Gift Card,Gift cards,USD,sell,US,UZ,1246.86,742.95418663


In [3]:
trades_paxful.sort_values(['amount', 'date'], inplace = True)

In [83]:
time_window = timedelta(hours=5)

trades_paxful = trades_paxful.set_index('date')
trades_paxful = trades_paxful.sort_index(ascending=False) #sort to be compatible with pd.rolling()
trades_paxful['N_i'] = trades_paxful['tid'].rolling(time_window).count() - 1

trades_paxful = trades_paxful.sort_index(ascending=True) #re-sort
trades_paxful = trades_paxful.reset_index()
trades_paxful['x_i'] = trades_paxful.groupby('amount').cumcount() #get count of trade size frequency
trades_paxful['i'] = trades_paxful.index + 1 #get trade number, ordered by time
trades_paxful['p_i'] = trades_paxful['x_i']/trades_paxful['i']
trades_paxful['theta'] = 1 - (1-trades_paxful['p_i'])**trades_paxful['N_i']

trades_paxful = trades_paxful.drop(columns=['N_i', 'i', 'p_i'])

trades_paxful

,date,tid,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,crypto_rate_usd,amount_usd,x_i,theta,N_i,i,p_i
0,2017-03-15 06:04:04,4544629,0.01306959,20.00,"Any Visa, MasterCard and AmEx Gift Cards",Gift cards,USD,buy,US,US,1246.86,16.29594899,0,0.00000000,1045.0,1,0.00000000
1,2017-03-15 06:04:26,4544627,0.05728689,100.00,PayPal,Online wallets,USD,buy,CM,US,1246.86,71.42873167,0,0.00000000,1044.0,2,0.00000000
2,2017-03-15 06:04:39,4544639,0.01688459,25.00,Amazon Gift Card,Gift cards,USD,buy,US,US,1247.49,21.06335718,0,0.00000000,1044.0,3,0.00000000
3,2017-03-15 06:05:27,4544613,0.09403504,150.00,iTunes Gift Card,Gift cards,USD,buy,CN,ZA,1246.86,117.24852997,0,0.00000000,1048.0,4,0.00000000
4,2017-03-15 06:05:28,4544617,0.59586015,1300.00,Amazon Gift Card,Gift cards,USD,sell,US,UZ,1246.86,742.95418663,0,0.00000000,1047.0,5,0.00000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89668395,2022-10-23 11:04:25,199258589,0.00145226,19600.00,International Wire Transfer (SWIFT),Bank transfers,NGN,sell,NG,NG,18967.26,27.54539301,133,0.00000000,0.0,89668396,0.00000148
89668396,2022-10-28 17:59:12,199372404,0.00426460,1450.03,Bank Transfer,Bank transfers,ZAR,sell,ZA,ZA,19394.79,82.71102143,71,0.00000158,2.0,89668397,0.00000079
89668397,2022-10-28 20:13:13,197433205,0.00069744,9600.00,Bank Transfer,Bank transfers,NGN,sell,NG,NG,19732.57,13.76228362,150,0.00000167,1.0,89668398,0.00000167
89668398,2022-10-28 20:14:42,197162038,0.00072850,10000.00,Bank Transfer,Bank transfers,NGN,sell,NG,NG,19801.36,14.42529076,181,0.00000000,0.0,89668399,0.00000202


In [84]:
trades_paxful = trades_paxful.drop(columns=['N_i', 'i', 'p_i'])

In [10]:

    
# # t = str(window) + 'h'

# matched_trades_paxful = trades_paxful[trades_paxful['x_i'] >= 2]

# matched_trades_paxful.sort_values(['amount', 'date'], inplace=True)
# matched_trades_paxful['tid2'] = matched_trades_paxful['tid'].shift(-1)
# matched_trades_paxful['date2'] = matched_trades_paxful['date'].shift(-1)
# matched_trades_paxful['amount2'] = matched_trades_paxful['amount'].shift(-1)
# matched_trades_paxful['currency2'] = matched_trades_paxful['currency'].shift(-1)
# matched_trades_paxful['type2'] = matched_trades_paxful['type'].shift(-1)
# matched_trades_paxful['payment_method2'] = matched_trades_paxful['payment_method'].shift(-1)
# matched_trades_paxful['time_diff'] = matched_trades_paxful['date2'] - matched_trades_paxful['date']    
# matched_trades_paxful['user_cc2'] = matched_trades_paxful['user_cc'].shift(-1)
# matched_trades_paxful = matched_trades_paxful[(matched_trades_paxful['time_diff'] <= (time_window)) & (matched_trades_paxful['amount'] == matched_trades_paxful['amount2']) & (matched_trades_paxful['theta'] <= 0.05)]
# matched_trades_paxful = matched_trades_paxful.sort_values('date')
# matched_trades_paxful


C:\Users\Austin\AppData\Local\Temp\ipykernel_2568\3545898142.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_trades_paxful.sort_values(['amount', 'date'], inplace=True)
C:\Users\Austin\AppData\Local\Temp\ipykernel_2568\3545898142.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_trades_paxful['tid2'] = matched_trades_paxful['tid'].shift(-1)
C:\Users\Austin\AppData\Local\Temp\ipykernel_2568\3545898142.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [11]:
matched_trades_paxful = matched_trades_paxful.sort_values('date')

In [12]:
matched_trades_paxful

,date,tid,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,...,x_i,theta,tid2,date2,amount2,currency2,type2,payment_method2,time_diff,user_cc2
46608,2017-03-21 06:55:25,4651459,0.01911914,25.0,Amazon Gift Card,Gift cards,USD,buy,TH,US,...,2,0.04986388,4651536.0,2017-03-21 06:57:57,0.01911914,USD,buy,Amazon Gift Card,0 days 00:02:32,US
46609,2017-03-21 06:55:29,4651449,0.03579687,50.0,iTunes Gift Card,Gift cards,USD,buy,JP,NG,...,2,0.04982206,4651455.0,2017-03-21 06:56:47,0.03579687,USD,buy,iTunes Gift Card,0 days 00:01:18,NG
46620,2017-03-21 06:58:19,4651519,0.01640904,25.0,iTunes Gift Card,Gift cards,USD,buy,IN,NG,...,2,0.04956599,4651515.0,2017-03-21 06:59:46,0.01640904,USD,buy,iTunes Gift Card,0 days 00:01:27,US
46629,2017-03-21 07:00:13,4651550,0.01638592,25.0,iTunes Gift Card,Gift cards,USD,buy,US,NG,...,2,0.04931204,4651556.0,2017-03-21 07:00:40,0.01638592,USD,buy,iTunes Gift Card,0 days 00:00:27,US
46636,2017-03-21 07:02:57,4651613,0.01636126,25.0,iTunes Gift Card,Gift cards,USD,buy,US,NG,...,2,0.04938636,4651673.0,2017-03-21 07:10:58,0.01636126,USD,buy,iTunes Gift Card,0 days 00:08:01,NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89667621,2022-09-29 06:02:29,200223486,0.00705863,200.0,iTunes Gift Card,Gift cards,USD,buy,US,NG,...,54,0.00023364,200223500.0,2022-09-29 06:02:31,0.00705863,USD,buy,Apple Gift Card (US only),0 days 00:00:02,US
89667620,2022-09-29 06:02:29,200223213,0.00705863,200.0,Apple Gift Card (US only),Gift cards,USD,buy,HK,PK,...,53,0.00022813,200223486.0,2022-09-29 06:02:29,0.00705863,USD,buy,iTunes Gift Card,0 days 00:00:00,NG
89667640,2022-09-29 06:03:40,200220373,0.00050596,1215.0,Bank Transfer,Bank transfers,KES,buy,KE,KE,...,187,0.00076716,200220390.0,2022-09-29 07:21:16,0.00050596,KES,buy,Bank Transfer,0 days 01:17:36,KE
89667644,2022-09-29 06:03:54,200223984,0.00047872,10.0,Dogecoin DOGE,Digital currencies,USD,buy,VN,VN,...,215,0.00087240,200223982.0,2022-09-29 06:04:00,0.00047872,USD,buy,Dogecoin DOGE,0 days 00:00:06,VN


In [13]:
path = '../temporary/matched_paxful_trades.csv'
matched_trades_paxful.to_csv(path, index = False)

In [26]:
matched_paxful_trades[matched_paxful_trades['currency'] != matched_paxful_trades['currency2']]

,tid,date,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,...,amount_usd,tid2,date2,amount2,currency2,time_diff,type2,payment_method2,user_cc2,advertiser_cc2
451,4545569,2017-03-15 07:50:58,0.03135484,50.00,iTunes Gift Card,Gift cards,USD,buy,US,US,...,39.22741323,4546157.0,2017-03-15 09:21:16,0.03135484,NGN,0 days 01:30:18,sell,Bank Transfer,NG,NG
661,4545969,2017-03-15 08:48:28,0.01566681,25.00,iTunes Gift Card,Gift cards,USD,buy,IN,US,...,19.61923283,4546167.0,2017-03-15 09:09:05,0.01566681,NGN,0 days 00:20:37,sell,Bank Transfer,US,NG
727,4546199,2017-03-15 09:10:09,0.05863582,95.00,iTunes Gift Card,Gift cards,USD,buy,US,NG,...,73.35927440,4546323.0,2017-03-15 10:27:22,0.05863582,NGN,0 days 01:17:13,sell,Bank Transfer,NG,KE
1031,4546861,2017-03-15 10:58:07,0.00230000,3.51,PayPal,Online wallets,AUD,sell,LK,AU,...,2.86918100,4547657.0,2017-03-15 12:39:40,0.00230000,NZD,0 days 01:41:33,sell,PayPal,AU,LK
1164,4547412,2017-03-15 11:53:39,0.03205478,50.00,iTunes Gift Card,Gift cards,USD,buy,US,GB,...,40.07071883,4547583.0,2017-03-15 13:28:41,0.03205478,NGN,0 days 01:35:02,sell,Bank Transfer,NG,NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89665709,200219803,2022-09-29 04:37:07,0.00146707,40.00,eBay Gift Card,Gift cards,USD,buy,VN,NG,...,28.67290021,200220087.0,2022-09-29 04:50:31,0.00146707,NGN,0 days 00:13:24,sell,Bank Transfer,NG,NG
89665729,200210153,2022-09-29 04:38:13,0.00162496,23184.00,Bank Transfer,Bank transfers,NGN,sell,NG,NG,...,31.69983343,200104810.0,2022-09-29 05:09:47,0.00162496,ZAR,0 days 00:31:34,sell,FNB E-WALLET,ZA,ZA
89665985,200220412,2022-09-29 04:50:16,0.00054547,7673.20,Bank Transfer,Bank transfers,NGN,sell,NG,NG,...,10.67491881,200221261.0,2022-09-29 05:07:54,0.00054547,KES,0 days 00:17:38,buy,M-Pesa,KE,KE
89666583,200221690,2022-09-29 05:17:13,0.00500000,70000.00,Bank Transfer,Bank transfers,NGN,sell,NG,GB,...,97.86030000,200211288.0,2022-09-29 07:00:28,0.00500000,USD,0 days 01:43:15,sell,NIP (NIBSS Instant Payment),NG,NG


# Code Testing

In [74]:
test_data = trades_paxful[0:100]
test_data['Count'] = test_data.groupby(['amount'])['tid'].transform('count')
test_data = test_data[test_data['Count'] > 8]
test_data

C:\Users\Austin\AppData\Local\Temp\ipykernel_4952\2196175266.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Count'] = test_data.groupby(['amount'])['tid'].transform('count')


,tid,date,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,crypto_rate_usd,amount_usd,Count
28677834,66169733,2019-10-29 11:10:42,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9442.47,0.04976182,11
28677870,66169792,2019-10-29 11:12:08,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9442.47,0.04976182,11
28677895,66169849,2019-10-29 11:13:08,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9429.92,0.04969568,11
28677941,66169908,2019-10-29 11:15:08,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9429.92,0.04969568,11
28677987,66169846,2019-10-29 11:17:14,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9429.92,0.04969568,11
28678063,66170244,2019-10-29 11:21:25,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9439.04,0.04974374,11
28678264,66170747,2019-10-29 11:30:21,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9440.06,0.04974912,11
28678412,66171011,2019-10-29 11:35:57,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9437.33,0.04973473,11
28678445,66171107,2019-10-29 11:37:02,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9424.44,0.04966680,11
28678519,66171091,2019-10-29 11:40:54,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9424.44,0.04966680,11


In [9]:
import itertools

In [79]:
matched_ids = set()
time_window = pd.Timedelta(hours=5)
matched_dict = {}

grouped_data = test_data.sort_values(['amount', 'date']).groupby('amount')
for _, group in grouped_data:
    for ind, row in group.iterrows():
        print(row['tid'])
        if row['tid'] in matched_ids:
            tmp = pd.DataFrame()
        else:
            tmp = group[(group['date'] >= row['date']) &
                                (group['date'] <= row['date'] + time_window) &
                                (group['tid'] != row['tid']) &
                                (~group['tid'].isin(matched_ids))]
        # matched_trades['matched_id'] = row['tid']
        if tmp.empty:
            pass
        else:
            matched_trade = tmp.iloc[0]
            matched_dict[row['tid']] = matched_trade
            matched_ids.add(matched_trade['tid'])

matched_trades = pd.DataFrame.from_dict(matched_dict, orient = 'index')

matched_trades = matched_trades.add_suffix('2').rename_axis('tid')
matched_trades



66169733
66169792
66169849
66169908
66169846
66170244
66170747
66171011
66171107
66171091
66171219
66786494
66787442
67185231
67185387
67185494
67185761
67185815
67186305
67186660
67186816
67187007
67187059
67187117
67187364
67187524
67187738
67952490
67952643
67952528
67952718
67952757
67952817
67952824
67952895
67952943
67953035
67953036
67953056
67953123


,tid2,date2,amount2,price2,payment_method2,payment_method_group2,currency2,type2,advertiser_cc2,user_cc2,crypto_rate_usd2,amount_usd2,Count2
tid,,,,,,,,,,,,,
66169733,66169792,2019-10-29 11:12:08,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9442.47,0.04976182,11
66169849,66169908,2019-10-29 11:15:08,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9429.92,0.04969568,11
66169846,66170244,2019-10-29 11:21:25,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9439.04,0.04974374,11
66170747,66171011,2019-10-29 11:35:57,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9437.33,0.04973473,11
66171107,66171091,2019-10-29 11:40:54,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9424.44,0.04966680,11
66786494,66787442,2019-11-02 17:43:36,0.00000532,10.0,Amazon Gift Card,Gift cards,USD,buy,VN,VN,9354.05,0.04976355,29
67185231,67185387,2019-11-05 14:49:28,0.00000532,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9346.08,0.04972115,29
67185494,67185761,2019-11-05 14:53:20,0.00000532,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9358.10,0.04978509,29
67185815,67186305,2019-11-05 15:00:54,0.00000532,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9345.44,0.04971774,29


In [80]:
test_data = test_data.set_index('tid')
test_data

,date,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,crypto_rate_usd,amount_usd,Count
tid,,,,,,,,,,,,
66169733,2019-10-29 11:10:42,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9442.47,0.04976182,11
66169792,2019-10-29 11:12:08,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9442.47,0.04976182,11
66169849,2019-10-29 11:13:08,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9429.92,0.04969568,11
66169908,2019-10-29 11:15:08,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9429.92,0.04969568,11
66169846,2019-10-29 11:17:14,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9429.92,0.04969568,11
66170244,2019-10-29 11:21:25,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9439.04,0.04974374,11
66170747,2019-10-29 11:30:21,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9440.06,0.04974912,11
66171011,2019-10-29 11:35:57,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9437.33,0.04973473,11
66171107,2019-10-29 11:37:02,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9424.44,0.04966680,11


In [81]:
matched_trades_paxful = test_data.join(matched_trades, how = 'inner')
matched_trades_paxful

,date,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,crypto_rate_usd,...,price2,payment_method2,payment_method_group2,currency2,type2,advertiser_cc2,user_cc2,crypto_rate_usd2,amount_usd2,Count2
tid,,,,,,,,,,,,,,,,,,,,,
66169733,2019-10-29 11:10:42,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9442.47,...,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9442.47,0.04976182,11
66169849,2019-10-29 11:13:08,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9429.92,...,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9429.92,0.04969568,11
66169846,2019-10-29 11:17:14,0.00000527,10.0,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9429.92,...,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9439.04,0.04974374,11
66170747,2019-10-29 11:30:21,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9440.06,...,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9437.33,0.04973473,11
66171107,2019-10-29 11:37:02,0.00000527,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9424.44,...,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9424.44,0.04966680,11
66786494,2019-11-02 17:34:18,0.00000532,10.0,Amazon Gift Card,Gift cards,USD,buy,VN,VN,9350.42,...,10.0,Amazon Gift Card,Gift cards,USD,buy,VN,VN,9354.05,0.04976355,29
67185231,2019-11-05 14:48:16,0.00000532,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9344.98,...,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9346.08,0.04972115,29
67185494,2019-11-05 14:50:16,0.00000532,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9346.08,...,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9358.10,0.04978509,29
67185815,2019-11-05 14:55:26,0.00000532,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9358.10,...,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9345.44,0.04971774,29


In [62]:
matched_trades[matched_trades['Count2'] > 10]

,tid2,date2,amount2,price2,payment_method2,payment_method_group2,currency2,type2,advertiser_cc2,user_cc2,crypto_rate_usd2,amount_usd2,Count2
66169733,66169792,2019-10-29 11:12:08,0.00000527,10.00,Pizza Hut Gift Card,Gift cards,USD,buy,VN,VN,9442.47,0.04976182,11
66786494,66787442,2019-11-02 17:43:36,0.00000532,10.00,Amazon Gift Card,Gift cards,USD,buy,VN,VN,9354.05,0.04976355,29
67162337,67162580,2019-11-05 09:45:07,0.00000534,10.00,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9317.35,0.04975465,19
67161673,67176744,2019-11-05 13:11:24,0.00000535,10.00,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9291.13,0.04970755,51
66413446,66413613,2019-10-31 14:35:07,0.00000536,10.00,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9276.78,0.04972354,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139053923,139053978,2021-05-18 04:04:35,0.00004507,222.00,Afriex,Online wallets,INR,buy,US,US,44834.77,2.02070308,21
139051808,139051813,2021-05-18 03:21:51,0.00004508,222.00,UPI Transfer,Bank transfers,INR,buy,US,US,44794.64,2.01934237,12
134539593,134540168,2021-04-15 19:49:52,0.00004524,227.00,Domestic Wire Transfer,Bank transfers,INR,buy,IN,IN,63367.35,2.86673891,11
136992663,136994079,2021-05-03 06:20:58,0.00004528,43662.76,Airtel Money,Online wallets,IDR,buy,KE,KE,58072.77,2.62953503,11


In [10]:
test_data['Count'] = test_data.groupby(['amount'])['tid'].transform('count')
test_data = test_data[test_data['Count'] > 1]

C:\Users\Austin\AppData\Local\Temp\ipykernel_20176\4150706913.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Count'] = test_data.groupby(['amount'])['tid'].transform('count')


In [20]:
from pandasql import sqldf
time_window = pd.Timedelta(hours=5)

time_window_hours = time_window.total_seconds() / 3600

query = """
SELECT t1.*, t2.*
FROM df t1
JOIN df t2
ON t1.amount = t2.amount
AND t1.id != t2.id
AND ABS(julianday(t1.date) - julianday(t2.date)) * 24 <= {time_window_hours}
"""

# query = f"""
# SELECT * FROM test_data
# """


matched_trades = sqldf(query, locals())

matched_trades

ObjectNotExecutableError: Not an executable object: '\nSELECT t1.*, t2.*\nFROM df t1\nJOIN df t2\nON t1.amount = t2.amount\nAND t1.id != t2.id\nAND ABS(julianday(t1.date) - julianday(t2.date)) * 24 <= {time_window_hours}\n'

In [7]:
merge_data = test_data[['tid', 'amount', 'user_cc', 'amount_usd', 'date']].add_suffix('2')
merge_data

,tid2,amount2,user_cc2,amount_usd2,date2
75567831,169427811,0.00000081,KE,0.03466432,2022-01-19 15:52:45
73473151,165201217,0.00000286,NG,0.13761030,2021-12-10 12:46:17
68343853,155153476,0.00000289,NG,0.13403956,2021-09-13 11:20:29
24647527,56497218,0.00000402,JP,0.04737341,2019-08-09 16:39:50
24719284,56682585,0.00000419,US,0.04743122,2019-08-10 21:04:14
...,...,...,...,...,...
29075942,67162830,0.00000534,VN,0.04975780,2019-11-05 09:48:13
29082103,67178306,0.00000534,VN,0.04970232,2019-11-05 13:30:41
29082144,67178369,0.00000534,VN,0.04970232,2019-11-05 13:31:52
29082243,67178548,0.00000534,VN,0.04966494,2019-11-05 13:34:13


In [ ]:
merged = 

In [11]:
test_data

,tid,date,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,crypto_rate_usd,amount_usd,Count
24719284,56682585,2019-08-10 21:04:14,0.00000419,10.0,ApplePay,Online wallets,USD,buy,US,US,11320.10,0.04743122,2
24719489,56683236,2019-08-10 21:08:31,0.00000419,10.0,ApplePay,Online wallets,USD,buy,US,US,11315.50,0.04741194,2
22807599,52158744,2019-07-02 18:27:40,0.00000453,9.0,Paysafecard,Debit/credit cards,EUR,buy,ES,ES,10632.47,0.04816509,3
22807645,52158813,2019-07-02 18:28:24,0.00000453,9.0,Paysafecard,Debit/credit cards,EUR,buy,ES,ES,10632.47,0.04816509,3
26107468,60103933,2019-09-08 00:09:38,0.00000453,10.0,Google Pay,Online wallets,USD,buy,US,NG,10485.63,0.04749990,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29075942,67162830,2019-11-05 09:48:13,0.00000534,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9317.94,0.04975780,8
29082103,67178306,2019-11-05 13:30:41,0.00000534,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9307.55,0.04970232,8
29082144,67178369,2019-11-05 13:31:52,0.00000534,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9307.55,0.04970232,8
29082243,67178548,2019-11-05 13:34:13,0.00000534,10.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,VN,VN,9300.55,0.04966494,8


In [13]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
import pandas as pd

# Assuming df is your DataFrame containing the transactions
# Make sure 'timestamp' column is in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort DataFrame by 'timestamp'
df = df.sort_values(by='timestamp')

# Define the time window for matching transactions (e.g., within 1 hour)
time_window = pd.Timedelta(hours=1)

# Group transactions by 'amount'
grouped = df.groupby('amount')

# Create an empty DataFrame to store matched transactions
matched_transactions = pd.DataFrame(columns=df.columns)

# Iterate over each group
for _, group in grouped:
    # Iterate over each transaction in the group
    for i, transaction in group.iterrows():
        # Select transactions within the time window
        nearby_transactions = group[(group['timestamp'] >= transaction['timestamp']) & 
                                    (group['timestamp'] <= transaction['timestamp'] + time_window)]
        # Exclude the current transaction itself
        nearby_transactions = nearby_transactions[nearby_transactions['id'] != transaction['id']]
        # Append matched transactions to the result DataFrame
        matched_transactions = pd.concat([matched_transactions, nearby_transactions])

# Reset index of the result DataFrame
matched_transactions = matched_transactions.reset_index(drop=True)

# Print the result
print(matched_transactions)
